In [30]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))
sampleSubmission <- read.csv(file.path("..", "data", "sampleSubmission.csv"))

In [6]:
dim(train.data)

[1]  708 4872

In [44]:
#delets dependent variables
train.dub <- train.data[!duplicated(as.list(train.data))]


#Separate the dataset in predictors and responses
V.data.x = data.frame(train.dub[, -which(names(train.data) == "VALENCE.PLEASANTNESS")])   #All the features 
V.data.y = train.data$VALENCE.PLEASANTNESS  #the response

In [45]:
dim(V.data.x)

[1]  708 2882

In [46]:
#data set with VALENCE.PLEASANTNESS + V.. (with low varinace)
library(tidymodels)
library(leaps)

#V.data = data frame with removed low variance features
length <- length(train.dub[1,])
V.data.x$Intensity = as.numeric(V.data.x$Intensity)
idx.low.var <- apply(V.data.x, 2, var) < 0.24 # identify pixels with low variance 
V.data <- data.frame(V.data.x)
V.data <- data.frame(V.data[,!idx.low.var])
V.data$VALENCE.PLEASANTNESS <- train.data$VALENCE.PLEASANTNESS


── Attaching packages ────────────────────────────────────── tidymodels 0.1.2 ──
✔ broom     0.7.2      ✔ recipes   0.1.15
✔ dials     0.0.9      ✔ rsample   0.0.8 
✔ dplyr     1.0.2      ✔ tibble    3.0.4 
✔ infer     0.5.3      ✔ tidyr     1.1.2 
✔ modeldata 0.1.0      ✔ tune      0.1.2 
✔ parsnip   0.1.4      ✔ workflows 0.2.1 
✔ purrr     0.3.4      ✔ yardstick 0.0.7 
── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard()     masks scales::discard()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::src()         masks Hmisc::src()
✖ recipes::step()      masks stats::step()
✖ dplyr::summarize()   masks Hmisc::summarize()
✖ parsnip::translate() masks Hmisc::translate()


In [74]:
dim(V.data)

[1]  708 1577

In [73]:
#Correlation on the data : enlever tous les features qui ont une corrélation supérieure à 0.9
library(Hmisc) 
library(corrplot)
library(caret)
library(corrplot)
library(plyr)


corrData = scale(V.data)  # normalize the data frame. This will also convert to a matrix. 
descrCor <- cor(corrData) # Access the correlation matrix (Pearson's coefficients)
#descrCor

highlyCorrelated <- findCorrelation(descrCor, cutoff=0.9)  #Renvoie les indices des features qui sont 
highlyCorCol <- colnames(V.data)[highlyCorrelated]
#highlyCorCol

UnCorrelatedData <- V.data[, -which(colnames(V.data) %in% highlyCorCol)]
#UnCorrelatedData
dim(UnCorrelatedData)

[1] 708 364

In [ ]:
#split the data set into a training set and a validation set
dim <- length(UnCorrelatedData[,1])
idx.validation <- sample(1:dim, dim/3)

train <- UnCorrelatedData[-idx.validation,]
validation <- UnCorrelatedData[idx.validation,]


In [89]:
#Try linear regression
lmMod <- lm(train$VALENCE.PLEASANTNESS ~ ., data=train)  # build the model
distPred <- predict(lmMod, validation)  # predict distance
MSE <- mean((distPred - validation$VALENCE.PLEASANTNESS)^2)

Warning message in predict.lm(lmMod, validation):
“prediction from a rank-deficient fit may be misleading”

[1] 7238682

In [90]:
library(leaps)

In [91]:
#forward selection 
nvmax = 50
models.forward <- regsubsets(VALENCE.PLEASANTNESS ~ ., data = train, nvmax = nvmax, method = 'forward')


Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
“60  linear dependencies found”

Reordering variables and trying again:


In [92]:
predict.regsubsets <- function(object,newdata,id,...){
form <- as.formula(object$call[[2]])
mat <- model.matrix(form,newdata)
coefi <- coef(object,id=id)
xvars <- names(coefi)
mat[,xvars]%*%coefi
}



In [93]:
#MSE of all selected models
for (i in 1:nvmax){
    p <- predict.regsubsets(models.forward, validation, id = i)
    MSE <- mean((p - validation$VALENCE.PLEASANTNESS)^2)
    print(MSE)
}


[1] 574.4654
[1] 583.8883
[1] 572.6534
[1] 575.9336
[1] 574.43
[1] 565.6416
[1] 564.4326
[1] 560.0872
[1] 559.4106
[1] 559.8461
[1] 552.0904
[1] 554.9741
[1] 555.5677
[1] 554.1021
[1] 562.0664
[1] 558.8103
[1] 576.2027
[1] 579.389
[1] 580.7213
[1] 579.5571
[1] 579.5264
[1] 578.3436
[1] 580.8111
[1] 576.1535
[1] 578.4499
[1] 580.8109
[1] 583.9607
[1] 586.1458
[1] 586.1264
[1] 584.07
[1] 584.9518
[1] 589.3683
[1] 588.4743
[1] 587.3372
[1] 591.5947
[1] 593.8715
[1] 593.8106
[1] 591.7948
[1] 594.4287
[1] 601.4251
[1] 598.2412
[1] 600.8691
[1] 600.6789
[1] 598.8773
[1] 604.0274
[1] 606.0289
[1] 616.221
[1] 617.3411
[1] 617.3481
[1] 617.3263
